In [50]:
#dependencies and database adapters
import pandas as pd
import numpy as np
import psycopg2
from psycopg2 import OperationalError
from psycopg2.extensions import register_adapter, AsIs

#to fix below problems with database queries involving integers
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)

def adapt_np_int64(np_int64):
    return AsIs(np_int64)

register_adapter(np.int64, adapt_np_int64)

In [51]:
#establishing connection for database adapter
def create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection

In [52]:
#connection credentials for PostgreSQL connection
connection = create_connection(
    "sql_challenge_db", "postgres", "$!Xt333n", "localhost", "5432"
)

Connection to PostgreSQL DB successful


In [53]:
#creating database adaptation function
def execute_query(connection, query):
    connection.autocommit = True
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Query executed successfully")
    except OperationalError as e:
        print(f"The error '{e}' occurred")

In [54]:
#creating tables through the database adapter function
create_titles_table = """
CREATE TABLE IF NOT EXISTS titles (
  id VARCHAR(5) PRIMARY KEY,
  title VARCHAR(50)
)
"""
execute_query(connection, create_titles_table)

create_employees_table = """
CREATE TABLE IF NOT EXISTS employees (
  id INTEGER PRIMARY KEY,
  title_id VARCHAR(5) REFERENCES titles(id),
  birthdate DATE,
  first_name VARCHAR(20),
  last_name VARCHAR(20),
  sex VARCHAR(1),
  hire_date DATE
)
"""
execute_query(connection, create_employees_table)


create_departments_table = """
CREATE TABLE IF NOT EXISTS departments (
  id VARCHAR(5) PRIMARY KEY,
  name VARCHAR(20)
)
"""
execute_query(connection, create_departments_table)


create_salaries_table = """
CREATE TABLE IF NOT EXISTS salaries (
  emp_id INTEGER PRIMARY KEY REFERENCES employees(id),
  salary INTEGER
)
"""
execute_query(connection, create_salaries_table)


create_dept_manager_table = """
CREATE TABLE IF NOT EXISTS dept_manager (
  emp_id INTEGER PRIMARY KEY REFERENCES employees(id),
  dept_id VARCHAR(4) REFERENCES departments(id)
)
"""
execute_query(connection, create_dept_manager_table)


create_dept_emp_table = """
CREATE TABLE IF NOT EXISTS dept_emp (
  emp_id INTEGER PRIMARY KEY REFERENCES employees(id),
  dept_id VARCHAR(4) REFERENCES departments(id)
)
"""
execute_query(connection, create_dept_emp_table)

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully


In [55]:
#database connection statements
connection.autocommit = True
cursor = connection.cursor()

#reading in CSVs as dataframes with pandas
titles_df = pd.read_csv('titles.csv')
emp_df = pd.read_csv('employees.csv')
depts_df = pd.read_csv('departments.csv')
sals_df = pd.read_csv('salaries.csv')
dept_man_df = pd.read_csv('dept_manager.csv')
dept_emp_df = pd.read_csv('dept_emp.csv')

#checking new dataframes
print(emp_df.head())


       id title_id   birthdate  first_name  last_name sex   hire_date
0  473302    s0001   7/25/1953    Hideyuki   Zallocco   M   4/28/1990
1  475053    e0002  11/18/1954       Byong  Delgrande   F    9/7/1991
2   57444    e0002   1/30/1958       Berry       Babb   F   3/21/1992
3  421786    s0001   9/28/1957       Xiong   Verhoeff   M  11/26/1987
4  282238    e0003  10/28/1952  Abdelkader    Baumann   F   1/18/1991


In [56]:
#adapting first three dataframes to tables, and printing when finished
insert_query = (f"INSERT INTO titles (id, title) VALUES (%s, %s) ON CONFLICT DO NOTHING")

for index, row in titles_df.iterrows():
    cursor.execute(insert_query, (row['title_id'], row['title']))

print('Finished with "titles" table.')


insert_query = (f"INSERT INTO employees (id, title_id, birthdate, first_name, last_name, sex, hire_date) VALUES (%s, %s, %s, %s, %s, %s, %s) ON CONFLICT DO NOTHING")

for index, row in emp_df.iterrows():
    cursor.execute(insert_query, (row['id'], row['title_id'], row['birthdate'], row['first_name'], row['last_name'], row['sex'], row['hire_date']))

print('Finished with "employees" table.')


insert_query = (f"INSERT INTO departments (id, name) VALUES (%s, %s) ON CONFLICT DO NOTHING")

for index, row in depts_df.iterrows():
    cursor.execute(insert_query, (row['dept_no'], row['dept_name']))

print('Finished with "departments" table.')




Finished with "titles" table.
Finished with "employees" table.
Finished with "departments" table.


In [57]:
#adapting next three dataframes to tables, and printing when finished
insert_query = (f"INSERT INTO salaries (emp_id, salary) VALUES (%s, %s) ON CONFLICT DO NOTHING")

for index, row in sals_df.iterrows():
    cursor.execute(insert_query, (row['emp_no'], row['salary']))

print('Finished with "salaries" table.')


insert_query = (f"INSERT INTO dept_manager (emp_id, dept_id) VALUES (%s, %s) ON CONFLICT DO NOTHING")

for index, row in dept_man_df.iterrows():
    cursor.execute(insert_query, (row['emp_no'], row['dept_no']))

print('Finished with "dept_manager" table.')


insert_query = (f"INSERT INTO dept_emp (emp_id, dept_id) VALUES (%s, %s) ON CONFLICT DO NOTHING")

for index, row in dept_emp_df.iterrows():
    cursor.execute(insert_query, (row['emp_no'], row['dept_no']))

print('Finished with "dept_emp" table.')

Finished with "salaries" table.
Finished with "dept_manager" table.
Finished with "dept_emp" table.
